<a href="https://colab.research.google.com/github/sarwar-ku/data_science/blob/main/boostrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Note: using Google CoLab


Part 5.5: Benchmarking Regularization Techniques
Quite a few hyperparameters have been introduced so far. Tweaking each of these values can have an effect on the score obtained by your neural networks. Some of the hyperparameters seen so far include:

Number of layers in the neural network
How many neurons in each layer
What activation functions to use on each layer
Dropout percent on each layer
L1 and L2 values on each layer
To try out each of these hyperparameters you will need to run train neural networks with multiple settings for each hyperparameter. However, you may have noticed that neural networks often produce somewhat different results when trained multiple times. This is because the neural networks start with random weights. Because of this it is necessary to fit and evaluate a neural network times to ensure that one set of hyperparameters are actually better than another. Bootstrapping can be an effective means of benchmarking (comparing) two sets of hyperparameters.

Bootstrapping is similar to cross-validation. Both go through a number of cycles/folds providing validation and training sets. However, bootstrapping can have an unlimited number of cycles. Bootstrapping chooses a new train and validation split each cycle, with replacement. The fact that each cycle is chosen with replacement means that, unlike cross validation, there will often be repeated rows selected between cycles. If you run the bootstrap for enough cycles, there will be duplicate cycles.

In this part we will use bootstrapping for hyperparameter benchmarking. We will train a neural network for a specified number of splits (denoted by the SPLITS constant). For these examples we use 100. We will compare the average score at the end of the 100. By the end of the cycles the mean score will have converged somewhat. This ending score will be a much better basis of comparison than a single cross-validation. Additionally, the average number of epochs will be tracked to give an idea of a possible optimal value. Because the early stopping validation set is also used to evaluate the the neural network as well, it might be slightly inflated. This is because we are both stopping and evaluating on the same sample. However, we are using the scores only as relative measures to determine the superiority of one set of hyperparameters to another, so this slight inflation should not present too much of a problem.

Because we are benchmarking, we will display the amount of time taken for each cycle. The following function can be used to nicely format a time span.

In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

Bootstrapping for Regression
Regression bootstrapping uses the ShuffleSplit object to perform the splits. This technique is similar to KFold for cross-validation; no balancing occurs. We will attempt to predict the age column for the jh-simple-dataset; the following code loads this data.

In [3]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generate dummies for job
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generate dummies for area
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Generate dummies for product
df = pd.concat([df,pd.get_dummies(df['product'],prefix="product")],axis=1)
df.drop('product', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convert to numpy - Classification
x_columns = df.columns.drop('age').drop('id')
x = df[x_columns].values
y = df['age'].values

The following code performs the bootstrap. The architecture of the neural network can be adjusted to compare many different configurations.

In [4]:
import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ShuffleSplit

SPLITS = 50

# Bootstrap
boot = ShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

# Loop through samples
for train, test in boot.split(x):
    start_time = time.time()
    num+=1

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construct neural network
    model = Sequential()
    model.add(Dense(20, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=5, verbose=0, mode='auto', restore_best_weights=True)

    # Train on the bootstrap sample
    model.fit(np.array(x_train).astype('float32'),np.array(y_train).astype('float32'),validation_data=(np.array(x_test).astype('float32'),np.array(y_test).astype('float32')),
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)

    # Predict on the out of boot (validation)
    pred = model.predict(np.array(x_test).astype('float32'))

    # Measure this bootstrap's log loss
    score = np.sqrt(metrics.mean_squared_error(pred,np.array(y_test).astype('float32')))
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)

    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f},"
          f" stdev={mdev:.6f}",
          f" epochs={epochs}, mean epochs={int(m2)}",
          f" time={hms_string(time_took)}")

7/7 [==============================] - 0s 6ms/step
#1: score=0.745393, mean score=0.745393, stdev=0.000000  epochs=112, mean epochs=112  time=0:00:29.81
7/7 [==============================] - 0s 3ms/step
#2: score=1.045139, mean score=0.895266, stdev=0.149873  epochs=121, mean epochs=116  time=0:00:22.43
7/7 [==============================] - 0s 3ms/step
#3: score=0.623238, mean score=0.804590, stdev=0.177254  epochs=129, mean epochs=120  time=0:00:18.11
7/7 [==============================] - 0s 3ms/step
#4: score=0.691162, mean score=0.776233, stdev=0.161172  epochs=117, mean epochs=119  time=0:00:21.44
7/7 [==============================] - 0s 2ms/step
#5: score=0.640754, mean score=0.749137, stdev=0.154006  epochs=114, mean epochs=118  time=0:00:21.36
7/7 [==============================] - 0s 2ms/step
#6: score=0.753098, mean score=0.749798, stdev=0.140596  epochs=133, mean epochs=121  time=0:00:21.44
7/7 [==============================] - 0s 3ms/step
#7: score=0.587712, mean score=

The bootstrapping process for classification is similar, and I present it in the next section.

Bootstrapping for Classification
Regression bootstrapping uses the StratifiedShuffleSplit class to perform the splits. This class is similar to StratifiedKFold for cross-validation, as the classes are balanced so that the sampling does not affect proportions. To demonstrate this technique, we will attempt to predict the product column for the jh-simple-dataset; the following code loads this data.

In [5]:
import pandas as pd
from scipy.stats import zscore

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generate dummies for job
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generate dummies for area
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convert to numpy - Classification
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Classification
products = dummies.columns
y = dummies.values

We now run this data through a number of splits specified by the SPLITS variable. We track the average error through each of these splits.

In [6]:
import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit

SPLITS = 50

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1,
                                random_state=42)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

# Loop through samples
for train, test in boot.split(x,df['product']):
    start_time = time.time()
    num+=1

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construct neural network
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(25, activation='relu')) # Hidden 2
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=25, verbose=0, mode='auto', restore_best_weights=True)

    # Train on the bootstrap sample
    #model.fit(x_train,y_train,validation_data=(x_test,y_test),
    #          callbacks=[monitor],verbose=0,epochs=1000)

    model.fit(np.array(x_train).astype('float32'),np.array(y_train).astype('float32'),validation_data=(np.array(x_test).astype('float32'),np.array(y_test).astype('float32')),
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)

    # Predict on the out of boot (validation)
    pred = model.predict(np.array(x_test).astype('float32'))

    # Measure this bootstrap's log loss
    y_compare = np.argmax(np.array(y_test).astype('float32'),axis=1) # For log loss calculation
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)

    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}," +\
          f"stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}," +\
          f" time={hms_string(time_took)}")

7/7 [==============================] - 0s 2ms/step
#1: score=0.674864, mean score=0.674864,stdev=0.000000, epochs=64, mean epochs=64, time=0:00:10.93
7/7 [==============================] - 0s 2ms/step
#2: score=0.673737, mean score=0.674300,stdev=0.000564, epochs=56, mean epochs=60, time=0:00:11.19
7/7 [==============================] - 0s 2ms/step
#3: score=0.659667, mean score=0.669423,stdev=0.006913, epochs=62, mean epochs=60, time=0:00:11.19
7/7 [==============================] - 0s 3ms/step
#4: score=0.678015, mean score=0.671571,stdev=0.007049, epochs=61, mean epochs=60, time=0:00:10.09
7/7 [==============================] - 0s 2ms/step
#5: score=0.632205, mean score=0.663698,stdev=0.016962, epochs=93, mean epochs=67, time=0:00:13.72
7/7 [==============================] - 0s 3ms/step
#6: score=0.686846, mean score=0.667556,stdev=0.017725, epochs=58, mean epochs=65, time=0:00:08.96
7/7 [==============================] - 0s 2ms/step
#7: score=0.726102, mean score=0.675919,stdev=0.0

Benchmarking
Now that we've seen how to bootstrap with both classification and regression, we can start to try to optimize the hyperparameters for the jh-simple-dataset data. For this example, we will encode for classification of the product column. Evaluation will be in log loss.

In [7]:
import pandas as pd
from scipy.stats import zscore

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generate dummies for job
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generate dummies for area
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],
               axis=1)
df.drop('area', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convert to numpy - Classification
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Classification
products = dummies.columns
y = dummies.values

I performed some optimization, and the code has the best settings that I could determine. Later in this book, we will see how we can use an automatic process to optimize the hyperparameters.

In [8]:
import pandas as pd
import os
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU

SPLITS = 100

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)

# Track progress
mean_benchmark = []
epochs_needed = []
num = 0

# Loop through samples
for train, test in boot.split(x,df['product']):
    start_time = time.time()
    num+=1

    # Split train and test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construct neural network
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation=PReLU(), \
        kernel_regularizer=regularizers.l2(1e-4))) # Hidden 1
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=PReLU(), \
        activity_regularizer=regularizers.l2(1e-4))) # Hidden 2
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=PReLU(), \
        activity_regularizer=regularizers.l2(1e-4)
    )) # Hidden 3
#    model.add(Dropout(0.5)) - Usually better performance
# without dropout on final layer
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=100, verbose=0, mode='auto', restore_best_weights=True)

    # Train on the bootstrap sample
    model.fit(np.array(x_train).astype('float32'),np.array(y_train).astype('float32'),validation_data=(np.array(x_test).astype('float32'),np.array(y_test).astype('float32')), \
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)

    # Predict on the out of boot (validation)
    pred = model.predict(np.array(x_test).astype('float32'))

    # Measure this bootstrap's log loss
    y_compare = np.argmax(np.array(y_test).astype('float32'),axis=1) # For log loss calculation
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)

    # Record this iteration
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f},"
          f"stdev={mdev:.6f}, epochs={epochs},"
          f"mean epochs={int(m2)}, time={hms_string(time_took)}")

7/7 [==============================] - 0s 3ms/step
#1: score=0.726197, mean score=0.726197,stdev=0.000000, epochs=127,mean epochs=127, time=0:00:43.29
7/7 [==============================] - 0s 3ms/step
#2: score=0.637087, mean score=0.681642,stdev=0.044555, epochs=214,mean epochs=170, time=0:00:43.01
7/7 [==============================] - 0s 2ms/step
#3: score=0.675545, mean score=0.679610,stdev=0.036493, epochs=242,mean epochs=194, time=0:00:50.18
7/7 [==============================] - 0s 6ms/step
#4: score=0.573676, mean score=0.653126,stdev=0.055704, epochs=247,mean epochs=207, time=0:00:53.80
7/7 [==============================] - 0s 2ms/step
#5: score=0.602901, mean score=0.643081,stdev=0.053721, epochs=183,mean epochs=202, time=0:00:38.31
7/7 [==============================] - 0s 2ms/step
#6: score=0.610514, mean score=0.637653,stdev=0.050520, epochs=151,mean epochs=194, time=0:00:31.60
7/7 [==============================] - 0s 3ms/step
#7: score=0.598932, mean score=0.632122,std